# Test 01: Descarga y Preprocesamiento de Imágenes

Este notebook valida:
1. Descarga del dataset SKU-110K
2. Preprocesamiento de imágenes
3. Visualización de resultados

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from loguru import logger

# Configurar logging
logger.remove()
logger.add(sys.stderr, level="INFO")

## 1. Importar módulos del proyecto

In [ ]:
from shelf_occupancy.config import load_config
from shelf_occupancy.utils import load_image, save_image
from shelf_occupancy.preprocessing import ImagePreprocessor

# Cargar configuración
config = load_config()
print("✅ Configuración cargada")
print(f"   - Target size: {config.preprocessing.target_size}")
print(f"   - CLAHE clip limit: {config.preprocessing.clahe.clip_limit}")

## 2. Descargar dataset (si es necesario)

In [ ]:
# Nota: Por ahora vamos a usar imágenes de ejemplo
# Si quieres descargar el dataset completo, ejecuta:
# !uv run python -m shelf_occupancy.data.download_dataset --n-samples 10

data_dir = Path("../data")
raw_dir = data_dir / "raw" / "sample"
processed_dir = data_dir / "processed"
results_dir = data_dir / "results"

# Crear directorios
raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)
results_dir.mkdir(parents=True, exist_ok=True)

print(f"📁 Directorios creados:")
print(f"   - Raw: {raw_dir}")
print(f"   - Processed: {processed_dir}")
print(f"   - Results: {results_dir}")

## 3. Crear imagen de prueba

Como el dataset SKU-110K es pesado, vamos a crear una imagen sintética para probar el pipeline.

In [ ]:
import cv2

def create_synthetic_shelf_image(width=800, height=600):
    """Crea una imagen sintética de anaquel para pruebas."""
    # Crear imagen base
    img = np.ones((height, width, 3), dtype=np.uint8) * 200
    
    # Agregar ruido
    noise = np.random.randint(-30, 30, (height, width, 3), dtype=np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    
    # Dibujar líneas horizontales (anaqueles)
    shelf_heights = [100, 250, 400, 550]
    for y in shelf_heights:
        cv2.line(img, (0, y), (width, y), (50, 50, 50), 3)
    
    # Dibujar líneas verticales (divisiones)
    for x in range(0, width, 150):
        cv2.line(img, (x, 0), (x, height), (60, 60, 60), 2)
    
    # Simular productos (rectángulos)
    colors = [(180, 100, 100), (100, 180, 100), (100, 100, 180), (180, 180, 100)]
    np.random.seed(42)
    
    for shelf_y in shelf_heights[:-1]:
        for i in range(5):
            x = np.random.randint(10, width - 80)
            y = shelf_y + np.random.randint(10, 100)
            w = np.random.randint(40, 80)
            h = np.random.randint(60, 120)
            color = colors[np.random.randint(0, len(colors))]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, -1)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 0), 2)
    
    # Agregar variación de iluminación
    overlay = np.zeros_like(img)
    cv2.circle(overlay, (width // 3, height // 3), 400, (255, 255, 255), -1)
    img = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)
    
    return img

# Crear y guardar imagen sintética
test_img = create_synthetic_shelf_image()
test_img_path = raw_dir / "test_shelf.jpg"
save_image(test_img, test_img_path)

print(f"✅ Imagen de prueba creada: {test_img_path}")

# Visualizar
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
plt.title("Imagen Sintética de Anaquel")
plt.axis('off')
plt.tight_layout()
plt.show()

## 4. Probar preprocesamiento

In [ ]:
# Cargar imagen
image = load_image(test_img_path, color_mode="BGR")
print(f"Imagen cargada: {image.shape}, dtype: {image.dtype}")

# Crear preprocesador
preprocessor = ImagePreprocessor(config.preprocessing)

# Aplicar cada paso individualmente para visualización
img_clahe = preprocessor.apply_clahe(image.copy())
img_filtered = preprocessor.apply_bilateral_filter(img_clahe.copy())
img_resized = preprocessor.resize(img_filtered.copy())

print(f"✅ Preprocesamiento completado")
print(f"   - CLAHE: {img_clahe.shape}")
print(f"   - Filtrado: {img_filtered.shape}")
print(f"   - Redimensionado: {img_resized.shape}")

## 5. Visualizar comparación

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original
axes[0, 0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0, 0].set_title('Original', fontsize=14, fontweight='bold')
axes[0, 0].axis('off')

# CLAHE
axes[0, 1].imshow(cv2.cvtColor(img_clahe, cv2.COLOR_BGR2RGB))
axes[0, 1].set_title('CLAHE (Corrección de Iluminación)', fontsize=14, fontweight='bold')
axes[0, 1].axis('off')

# Filtrado bilateral
axes[1, 0].imshow(cv2.cvtColor(img_filtered, cv2.COLOR_BGR2RGB))
axes[1, 0].set_title('Filtrado Bilateral (Reducción de Ruido)', fontsize=14, fontweight='bold')
axes[1, 0].axis('off')

# Redimensionado
axes[1, 1].imshow(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'Redimensionado {img_resized.shape[:2][::-1]}', fontsize=14, fontweight='bold')
axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig(results_dir / 'preprocessing_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✅ Comparación guardada en: {results_dir / 'preprocessing_comparison.png'}")

## 6. Guardar imagen preprocesada

In [ ]:
# Guardar versión procesada
processed_path = processed_dir / "test_shelf_processed.jpg"
save_image(img_resized, processed_path)

print(f"✅ Imagen procesada guardada: {processed_path}")

## 7. Análisis de histogramas

In [ ]:
def plot_histogram_comparison(img1, img2, title1="Original", title2="Procesada"):
    """Compara histogramas de dos imágenes."""
    fig, axes = plt.subplots(2, 3, figsize=(15, 8))
    
    colors = ('b', 'g', 'r')
    titles = ['Blue', 'Green', 'Red']
    
    for i, (color, title) in enumerate(zip(colors, titles)):
        # Histograma imagen 1
        hist1 = cv2.calcHist([img1], [i], None, [256], [0, 256])
        axes[0, i].plot(hist1, color=color)
        axes[0, i].set_title(f'{title1} - {title}', fontweight='bold')
        axes[0, i].set_xlim([0, 256])
        axes[0, i].grid(alpha=0.3)
        
        # Histograma imagen 2
        hist2 = cv2.calcHist([img2], [i], None, [256], [0, 256])
        axes[1, i].plot(hist2, color=color)
        axes[1, i].set_title(f'{title2} - {title}', fontweight='bold')
        axes[1, i].set_xlim([0, 256])
        axes[1, i].grid(alpha=0.3)
    
    plt.tight_layout()
    return fig

fig = plot_histogram_comparison(image, img_clahe, "Original", "CLAHE")
plt.savefig(results_dir / 'histogram_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Comparación de histogramas completada")

## ✅ Resumen

En este notebook hemos validado:

1. ✅ Carga de configuración desde YAML
2. ✅ Creación de imagen sintética de anaquel
3. ✅ Preprocesamiento con CLAHE (corrección de iluminación)
4. ✅ Filtrado bilateral (reducción de ruido)
5. ✅ Redimensionamiento de imágenes
6. ✅ Visualización de resultados
7. ✅ Análisis de histogramas

**Próximos pasos:**
- Implementar detección de bordes y líneas
- Integrar Depth-Anything-V2 para estimación de profundidad
- Desarrollar análisis de ocupación por cuadrículas